In [80]:
from numpy import *
import operator

## 香农信息熵公式

$H(x)=-\sum_{i=1}^n{p_ilog_2{p_i}}$

其中 $p_i$ 为某一事件出现的概率，对数 $-log_2{p_i}$ 即是该事件的信息量（概率越小信息量越大，且非负；以 $2$ 为底是因为信息在计算机中由二进制表示）。

显然，$H(x)$ 即为一随机给定的数据中包含的信息量的期望值

In [72]:
def entropy(dset, idx):
    total = len(dset)
    labels = array(dset)[:, idx]
    count = {}
    for label in labels:
        count[label] = count.get(label, 0) + 1
    # 计算每一个 label 出现的概率
    p = array(list(count.values())) / total
    
#     print(total, count, p)
    # 香农公式计算信息熵    
    shannonEnt = sum(p*-log2(p))
    return shannonEnt

In [3]:
entropy([
    [1,2,3,'a'],
    [1,2,3,'b'],
    [1,2,3,'c'],
    [1,2,3,'d'],
    [1,2,3,'a'],
    [1,2,3,'a'],
    [1,2,3,'c'],
    [1,2,3,'d'],
    [1,2,3,'d'],
], -1)

9 {'a': 3, 'b': 1, 'c': 2, 'd': 3} [0.33333333 0.11111111 0.22222222 0.33333333]


1.8910611120726526

## 特征选取

In [25]:
def group(dset, idx):
    feat = array(dset)[:, idx]
    feat_set = set(feat)
    groups = {}
    for row in dset:
        group = groups.get(row[idx], None)
        if group is None:
            group = []
            groups[row[idx]] = group
        group.append(row)
    return groups

In [88]:
sub = group([
    [1,2,'yes'],
    [2,1,'no'],
    [1,1,'yes'],
    [2,2,'no']
], 2)

In [42]:
sub = group([
    [1,2,1,'a'],
    [1,2,2,'b'],
    [1,2,3,'c'],
    [1,2,4,'d'],
    [1,2,1,'a'],
    [1,2,2,'a'],
    [1,2,3,'c'],
    [1,2,4,'d'],
    [1,2,4,'d'],
], 2)

In [89]:
sub

{'yes': [[1, 2, 'yes'], [1, 1, 'yes']], 'no': [[2, 1, 'no'], [2, 2, 'no']]}

In [56]:
list(sub.values())

[[[1, 2, 1, 'a'], [1, 2, 1, 'a']],
 [[1, 2, 2, 'b'], [1, 2, 2, 'a']],
 [[1, 2, 3, 'c'], [1, 2, 3, 'c']],
 [[1, 2, 4, 'd'], [1, 2, 4, 'd'], [1, 2, 4, 'd']]]

In [45]:
for sub_dset in sub.values():
    print(entropy(sub_dset, -1))

2 {'a': 2} [1.]
0.0
2 {'b': 1, 'a': 1} [0.5 0.5]
1.0
2 {'c': 2} [1.]
0.0
3 {'d': 3} [1.]
0.0


## 选取最优特征

遍历数据集 $dset$ 的每个特征，按特征值对数据集进行分组，并分别计算这些子集的信息熵 $subent_i$，每个子集出现的概率 $p_i=\frac {count(subset)} {count(dset)}$

若原数据集的信息熵为 $ent$，则有信息增益 $gain= ent- newent=ent-\sum {p_i}{subent_i}\$

信息增益越高，意味着经过此次划分后剩余的信息熵越小（即混乱程度越小，答案越容易确认），所以这种划分方式相对也就越好

In [85]:
def choose_feature(dset):
    # 特征数量
    total = len(dset)
    ent = entropy(dset, -1)
    features_count = len(dset[0]) - 1
    print('origin entropy: %.4f, feature count: %d' % (ent, features_count))
    
    if features_count <= 0:
        pass

    gains = {}
    for i in range(features_count):
        splited_ents = []
        sub_sets = group(dset, i)
        for sub_set in sub_sets.values():
            splited_ents.append(entropy(sub_set, -1))
        p = array(list(map(lambda s: len(s), sub_sets.values()))) / total
        new_ent = sum(p*splited_ents)
        gain = ent - new_ent
        gains[i] = gain
        print('if choose column index %d as feautre, new entropy is: %.4f. gain is: %.4f' % (i, new_ent, gain))
    return sorted(gains.items(), key=operator.itemgetter(1), reverse=True)[0]  # 根据增益由大到小排列

In [87]:
feat = choose_feature([
    [1,2,'yes'],
    [2,1,'no'],
    [1,1,'yes'],
    [2,2,'no']
])

print('optimized feature is column %d, info gain is %.4f' % feat)

origin entropy: 1.0000, feature count: 2
if choose column index 0 as feautre, new entropy is: 0.0000. gain is: 1.0000
if choose column index 1 as feautre, new entropy is: 1.0000. gain is: 0.0000
optimized feature is column 0, info gain is 1.0000
